# Chapter 04: 도구 사용 (Tool Use)

이 노트북에서는 LLM에 도구를 바인딩하고 활용하는 방법을 학습합니다.

## 주요 내용
- 계산기 도구 정의 및 사용
- Wikipedia 검색 도구 통합
- 도구 호출 결과 처리

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch04_tool_use.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langchain-community openai wikipedia python-dotenv


## 2. API 키 설정


In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"
    print("⚠️ API 키를 직접 입력해주세요.")


## 3. 계산기 도구 사용

LLM에 수학 연산 도구를 바인딩하여 정확한 계산을 수행합니다.


In [ ]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, ToolMessage

# 도구 정의
@tool
def multiply(x: float, y: float) -> float:
    """'x'와 'y'를 곱합니다."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """'x'를 'y'제곱합니다."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """'x'와 'y'를 더합니다."""
    return x + y

tools = [multiply, exponentiate, add]
print("✅ 계산기 도구가 정의되었습니다:", [t.name for t in tools])


In [ ]:
# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# 계산 질문
query = "393 * 12.25는 얼마인가요? 그리고 11 + 49는요?"
messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

print("🔧 도구 호출 결과:")
print("-" * 40)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "exponentiate": exponentiate}[tool_call["name"]]
    result = selected_tool.invoke(tool_call['args'])
    
    print(f"도구: {tool_call['name']}")
    print(f"인자: {tool_call['args']}")
    print(f"결과: {result}")
    print()
    
    tool_msg = ToolMessage(content=str(result), tool_call_id=tool_call["id"])
    messages.append(tool_msg)


In [ ]:
# 최종 응답 생성
final_response = llm_with_tools.invoke(messages)
print("🤖 최종 응답:")
print(final_response.content)


## 4. Wikipedia 검색 도구

외부 지식 소스인 Wikipedia를 검색하여 정보를 가져옵니다.


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Wikipedia 도구 설정
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# LLM에 Wikipedia 도구 바인딩
llm_with_wiki = llm.bind_tools([wiki_tool])

print("✅ Wikipedia 도구가 설정되었습니다.")


In [ ]:
# Wikipedia 검색 테스트
wiki_messages = [HumanMessage("Buzz Aldrin의 주요 업적은 무엇인가요?")]

ai_msg = llm_with_wiki.invoke(wiki_messages)
wiki_messages.append(ai_msg)

print("🔍 Wikipedia 검색 결과:")
print("-" * 40)
for tool_call in ai_msg.tool_calls:
    tool_msg = wiki_tool.invoke(tool_call)
    print(f"검색어: {tool_call['args']}")
    print(f"결과: {tool_msg.content[:500]}...")
    wiki_messages.append(tool_msg)

# 최종 응답
final_wiki_response = llm_with_wiki.invoke(wiki_messages)
print("\n🤖 최종 응답:")
print(final_wiki_response.content)
